## PyTorch 基本

* Tensor(张量)是PyTorch基本对象, 表示多维matrix,与NumPy中ndarray可以互换. 且使用方法类似

In [14]:
import torch
px = range(15)
tx = torch.Tensor(px) #从已有list创建Tensor
tx = tx.reshape(5,3)
tx

tensor([[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.],
        [12., 13., 14.]])

In [15]:
import numpy
nx = tx.numpy() #从tensor转换为numpy
nx

array([[ 0.,  1.,  2.],
       [ 3.,  4.,  5.],
       [ 6.,  7.,  8.],
       [ 9., 10., 11.],
       [12., 13., 14.]], dtype=float32)

In [16]:
ty = torch.from_numpy(nx) #从numpy转化为tensor
ty

tensor([[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.],
        [12., 13., 14.]])

* 以下Variable已被停用

>* Variable是Tensor的封装, 用于放入计算图中进行向前转播, 反向传播和自动求导. Variable有三个基本属性, data:包含的Tensor数据部分; grad传播方向的梯度,这个属性是延迟分配的且只允许进行一次; creator:创建这个Variable的Function引用,用于回溯整个链路,若是用户创建的Variable,creator为None,同时这种Variable称作Leaf Variable.autograd只会给Leaf Variable分配梯度.

>* Tensor 不能反向传播, Variable可以反向传播

In [22]:
x = torch.rand(4)
var = torch.variable(x,requires_grad = true)


AttributeError: module 'torch' has no attribute 'variable'

### 模型的保存与加载

torch.save(model,'model.pkl')  #保存model,''内为名字

model = torch('model.pkl')   #加载模型

torch.save(model.state_dict(),'params.pkl')  #保存网络参数

model.load_state_dict(torch.load('params.pkl')  #加载网络参数

###### 在torchvision.models中, PyTorch提供了常用模型:
* AlexNet
* VGG
* ResNet
* SqueezeNet
* DenseNet
* Inception v3

可以使用torch.util.model_zoo来预加载, 具体通过设置参数pretrained = True来实现

 只加载模型，不加载预训练参数 如果只需要网络结构，不需要训练模型的参数来初始化，可以将pretrained = False(default)

In [ ]:
import torchvision.models as models
resnet18 = models.resnet18(pretrained=True)
alexnet = models.alexnet(pretrained=True)
squeezenet = models.squeezenet1_0(pretrained=True)
vgg16 = models.vgg16(pretrained=True)
densenet = models.densenet161(pretrained=True)
inception = models.inception_v3(pretrained=True)

In [3]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
print(device)

cuda:0


## Dataset使用

In [2]:
from torch.utils.data import Dataset #导入Dataset类

In [3]:
help(Dataset)

Help on class Dataset in module torch.utils.data.dataset:

class Dataset(typing.Generic)
 |  An abstract class representing a :class:`Dataset`.
 |  
 |  All datasets that represent a map from keys to data samples should subclass
 |  it. All subclasses should overwrite :meth:`__getitem__`, supporting fetching a
 |  data sample for a given key. Subclasses could also optionally overwrite
 |  :meth:`__len__`, which is expected to return the size of the dataset by many
 |  :class:`~torch.utils.data.Sampler` implementations and the default options
 |  of :class:`~torch.utils.data.DataLoader`.
 |  
 |  .. note::
 |    :class:`~torch.utils.data.DataLoader` by default constructs a index
 |    sampler that yields integral indices.  To make it work with a map-style
 |    dataset with non-integral indices/keys, a custom sampler must be provided.
 |  
 |  Method resolution order:
 |      Dataset
 |      typing.Generic
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __add__(self, oth

In [4]:
import os
import cv2

class myData(Dataset): #继承Dataset类
    
    def __init__(self,data_dir,label_dir): #初始化函数用于加载数据集
        self.data_dir = data_dir
        self.label_dir = label_dir
        self.img_path_list = os.listdir(self.data_dir) #不必在此直接读取数据, 减少内存负担
        self.label_path_list = os.listdir(self,label_dir)
        
    def __getitem__(self,idx): #重写__getitem__方法, 用于返回idx索引所在的单个的sample和label
        img_name = self.img_path_list[idx]
        label_name = self.label_path_list[idx]
        img_item_path = os.path.join(self.data_dir,self.img_name) #加载图像的路经
        label_item_path = os.path.join(self.label_dir,self.label_name) #加载label路径
        img = cv2.imread(img_item_path)    #读取图片
        with open(label_item_path) as f:  #读取标签
            label = f.read()
        return img,label 
    def __len__(self): #重写__len__方法, 用于返回数据集长度
        return len(self.img_path_list)

## TensorBoard 使用

* writer.add_scalar()方法绘制标量图像

In [7]:
from torch.utils.tensorboard import SummaryWriter #导入tensorboard的summarywriter类


writer = SummaryWriter('logs') #创建实例,存储event在logs文件夹

# writer.add_image() #添加图片
for each in range(100):
    writer.add_scalar('y=2x',2*each,each) #添加标量
writer.close()
'''def add_scalar(
        self,
        tag,            #图像的title
        scalar_value,   #图像的y轴数据
        global_step=None, #图像的x轴数据
        walltime=None,  
        new_style=False,
        double_precision=False,
    )

    Args:
            tag (string): Data identifier
            scalar_value (float or string/blobname): Value to save
            global_step (int): Global step value to record
            walltime (float): Optional override default walltime (time.time())
              with seconds after epoch of event
            new_style (boolean): Whether to use new style (tensor field) or old
              style (simple_value field). New style could lead to faster data loading.'''

上述命令将创建tensorboard的一个event

打开tensorboard

用terminal进入logs父路径, 输入**tensorboard --logdir=logs --port=6007**(可选, 默认打开一个主机网页, 端口号为6006,此处可更改端口号)即可打开tensorboard

* writer.add_image()绘制单个图像图像, 使用 writer.add_images()绘制多个图像, 其中,图片参数数组为4维度,某张×高×宽×channel, step仍需循环加入(多次图片组读取)

In [ ]:
from torch.utils.tensorboard import SummaryWriter #导入tensorboard的summarywriter类
import cv2

writer = SummaryWriter('logs') #创建实例,存储event在logs文件夹

img_list = ['bue_78.png','img_002.jpg']
i = 1
for each in img_list:
  img = cv2.imread(each)
  writer.add_image('imgtitle',img,i,dataformats='HWC')
  i = i + 1
  print(i)
writer.close()

'''def add_image(
        self,
         tag,   #加载窗口的title
         img_tensor, #图片数组数据
         global_step=None, #数据步骤,若title相同, 可通过更改step,使之产生一定区别
          walltime=None, 
          dataformats="CHW" #图片数组形状, 3(RGB)*640(高)*720(宽), 可以通过令为HWC变成640*720*3加载. opencv为HWC的numpy.array形
    ):
        """Add image data to summary.

        Note that this requires the ``pillow`` package.

        Args:
            tag (string): Data identifier
            img_tensor (torch.Tensor, numpy.array, or string/blobname): Image data
            global_step (int): Global step value to record
            walltime (float): Optional override default walltime (time.time())
              seconds after epoch of event
            dataformats (string): Image data format specification of the form
              CHW, HWC, HW, WH, etc.
        Shape:
            img_tensor: Default is :math:`(3, H, W)`. You can use ``torchvision.utils.make_grid()`` to
            convert a batch of tensor into 3xHxW format or call ``add_images`` and let us do the job.
            Tensor with :math:`(1, H, W)`, :math:`(H, W)`, :math:`(H, W, 3)` is also suitable as long as
            corresponding ``dataformats`` argument is passed, e.g. ``CHW``, ``HWC``, ``HW``.
            '''

* 输出神经网络结构

In [ ]:
# writer.add_graph(model, input_to_model=None, verbose=False, use_strict_trace=True)
'''
    Args:
        model (torch.nn.Module) = Model to draw. #需要画出结构图的模型实例
        input_to_model (torch.Tensor or list of torch.Tensor) = A variable or a tuple of variables to be fed. #输入神经网络的输入Tensor
        verbose (bool) = Whether to print graph structure in console. #是否在Terminal输出图像
        use_strict_trace (bool) = Whether to pass keyword argument strict to torch.jit.trace. 
                            Pass False when you want the tracer to record your mutable container types (list, dict)
'''

## DataLoader使用

In [ ]:
from torch.utils.data import DataLoader
dataset = 'xx'
test_loader = DataLoader(dataset,batch_size=4,shuffle=True,drop_last=True) #通过DataLoader实例载入Dataset

for epoch in range(2): #加载两轮数据,可以在轮数内进行ANN训练
  for data in test_loader: #从test_loader里面加载数据包,
                            #data是一个list, 里面有imgs和labels 
                            #imgs是一个4维数组, labels是一个list,里面写有相应的sample的label
    imgs,labels = data 
    #可以在此对imgs和labels做一些事情

''' Args:
        dataset (Dataset): dataset from which to load the data.   #要加载的数据集
        batch_size (int, optional): how many samples per batch to load   #一次加载几张作为一个包,default= 1
            (default: ``1``).
        shuffle (bool, optional): set to ``True`` to have the data reshuffled  #是否打乱从数据集里面加载的顺序
            at every epoch (default: ``False``).
        sampler (Sampler or Iterable, optional): defines the strategy to draw
            samples from the dataset. Can be any ``Iterable`` with ``__len__``
            implemented. If specified, :attr:`shuffle` must not be specified.
        batch_sampler (Sampler or Iterable, optional): like :attr:`sampler`, but
            returns a batch of indices at a time. Mutually exclusive with
            :attr:`batch_size`, :attr:`shuffle`, :attr:`sampler`,
            and :attr:`drop_last`.
        num_workers (int, optional): how many subprocesses to use for data        #加载的子进程数, windows有bug只能为0,只有主进程
            loading. ``0`` means that the data will be loaded in the main process.
            (default: ``0``)
        collate_fn (callable, optional): merges a list of samples to form a
            mini-batch of Tensor(s).  Used when using batched loading from a
            map-style dataset.
        pin_memory (bool, optional): If ``True``, the data loader will copy Tensors
            into device/CUDA pinned memory before returning them.  If your data elements
            are a custom type, or your :attr:`collate_fn` returns a batch that is a custom type,
            see the example below.
        drop_last (bool, optional): set to ``True`` to drop the last incomplete batch,      #如果最后一个包不满足指定的数据数目,是否舍弃掉最后一个包
            if the dataset size is not divisible by the batch size. If ``False`` and
            the size of dataset is not divisible by the batch size, then the last batch
            will be smaller. (default: ``False``)
        timeout (numeric, optional): if positive, the timeout value for collecting a batch
            from workers. Should always be non-negative. (default: ``0``)
        worker_init_fn (callable, optional): If not ``None``, this will be called on each
            worker subprocess with the worker id (an int in ``[0, num_workers - 1]``) as
            input, after seeding and before data loading. (default: ``None``)
        generator (torch.Generator, optional): If not ``None``, this RNG will be used
            by RandomSampler to generate random indexes and multiprocessing to generate
            `base_seed` for workers. (default: ``None``)
        prefetch_factor (int, optional, keyword-only arg): Number of batches loaded
            in advance by each worker. ``2`` means there will be a total of
            2 * num_workers batches prefetched across all workers. (default: ``2``)
        persistent_workers (bool, optional): If ``True``, the data loader will not shutdown
            the worker processes after a dataset has been consumed once. This allows to
            maintain the workers `Dataset` instances alive. (default: ``False``)
        pin_memory_device (str, optional): the data loader will copy Tensors
            into device pinned memory before returning them if pin_memory is set to true.
'''

## nn骨架搭建

In [1]:
from torch import nn
import torch

class Model(nn.Module):  #创建神经网络骨架, 一定要重写__init__和forward函数

    def __init__(self):
        super(Model,self).__init__() #继承父类
        #在下做一些神经网络基础设定
        #xxxxx
        #xxxxx
    
    def forward(self, input):   #前向算法布置
        #布置前向算法
        output = '前向算法处理以后的input'
        return output

ann = Model()   #创建神经网络实例
input = torch.Tensor([1.0,2.0]) #传入Tensor型输入
output = ann(input) #前向算法通过实例直接调用

print(output) 


前向算法处理以后的input


## 卷积

从一个输入的张量(大的高和宽), 与kernel张量(小的高和宽)从头开始相匹配, 对应元素相乘并全部相加得到一个标量. 然后这个kernel再进行移动, 与input张量的其他数相匹配, 得到其他标量, 直到取完input张量

每一个Kernel相当于是图片在某一个维度上的特征, 训练模型将自动提取这一特征而不需要人手动提取

In [2]:
import torch
import torch.nn.functional as F


#设置2d卷积操作
# torch.nn.functional.conv2d(input, weight, bias=None, stride=1, padding=0, dilation=1, groups=1)
'''    Args:
        input = input tensor of shape (minibatch ,in_channels , iH , iW) #输入张量需要四个size, minibatch:子集个数, in_channels:通道个数
                                                                            iH:张量的高, iW:张量的宽
        weight = filters of shape (out_channels, in_channels/groups ,kH ,kW) #输入卷积核kernel, out_channels:输出通道个数, 
                                                                            in_channels/groups:输入通道个数(groups一般为1),
                                                                             kH:张量的高, kW:张量的宽
        bias = optional bias tensor of shape(out_channel) #偏置
        stride = the stride of the convolving kernel. Can be a single number or a tuple (sH, sW). Default: 1 #卷积核一次移动多少数据,
                                                                                            可以通过标量或者元组指定.默认为1
        padding = implicit paddings on both sides of the input.Can be a string {'valid', 'same'}, #扩充输入张量的每一个高宽维度
                    single number or a tuple (padH, padW). Default: 0                             #输入标量或者高宽元组
                    padding='valid' is the same as no padding.                                   
                    padding='same' pads the input so the output has the same shape as the input.
                    However, this mode doesn't support any stride values other than 1.    
        dilation = the spacing between kernel elements. Can be a single number or a tuple (dH, dW). Default: 1 #kernel中的元素间隔, 
                                                                                                                用于空洞卷积
        groups = split input into groups, in_channels should be divisible by the number of groups. Default: 1                                                                        
    '''
input = torch.tensor([[0,1,2,3,4],
                      [3,2,1,0,4],
                      [4,3,2,1,0],
                      [2,1,3,4,0],
                      [3,2,1,4,0]]) #输入张量为(5,5)型
kernel = torch.tensor([[0,1,2],
                       [1,2,0],
                       [2,0,1]])#kernel为(3,3)型
#以下重置两个张量为合法形式
input = torch.reshape(input,(1,1,5,5))
kernel = torch.reshape(kernel,(1,1,3,3))
#end

output = F.conv2d(input,kernel,stride=1) #2d卷积
print(output)

tensor([[[[22, 19, 16],
          [21, 14, 18],
          [18, 19, 14]]]])


*卷积实例*

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import cv2

from torch.utils.tensorboard import SummaryWriter
class Model(nn.Module): #定义卷积神经网络

    def __init__(self):
        super(Model,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=6,kernel_size=3,stride=1,padding=0)#设置卷积层相应参数

    def forward(self,input):#设置前向算法 
        output = self.conv1(input)
        return output

ann = Model() #实例化神经网络
img_path = 'bue_78.png'
img = cv2.imread(img_path)
img_chw = np.transpose(img,(2,0,1)) #将HWC格式的ndarrary图片,转化成CHW型, 数字代表索引将要变化成为的值 
img_chw = img_chw[np.newaxis] #给CHW型图片加一个维度, 使之符合卷积格式要求
input = torch.from_numpy(img_chw).float() #卷积需要数据为浮点型
writer = SummaryWriter('logs') #加载tensorborad可视化图像
output = ann(input) #调用神经网络前向算法, 得到卷积后的图像
print(output.shape)
output = torch.reshape(output,(-1,3,838,800))
writer.add_image('input',img,dataformats='HWC')
writer.add_image('output1',output[0])
writer.add_image('output2',output[1])
writer.close()

## 最大池化

对于一个Tensor, 使用最大池化核kernel去从头开始匹配, 取得这个kernel覆盖下的tensor最大值标量, 存为output里的一个元素, 直到取完.

*池化实例*

In [ ]:
import torch
import torch.nn as nn
import cv2
import numpy as np
from torch.utils.tensorboard import SummaryWriter

#nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
'''
    Args:
        kernel_size = the size of the window to take a max over #池化核的大小
        stride = the stride of the window. Default value is kernel_size #kernel步进大小, 默认为kernel的大小
        padding = implicit zero padding to be added on both sides #扩张原Tensor
        dilation = a parameter that controls the stride of elements in the window #kernel的空洞
        return_indices = if True, will return the max indices along with the outputs. Useful for torch.nn.MaxUnpool2d later
        ceil_mode = when True, will use ceil instead of floor to compute the output shape #是否在池化核越出Tensor范围时候继续池化剩下的
''' 

class Model(nn.Module):
    
    def __init__(self):
        super(Model,self).__init__()
        self.pool = nn.MaxPool2d(kernel_size=128,stride=3,padding=0,ceil_mode=True)


    def forward(self,input):
        output = self.pool(input)
        return output

ann = Model()
img_path = 'bue_78.png'
img = cv2.imread(img_path)
img_chw = np.transpose(img,(2,0,1)) #将HWC格式的ndarrary图片,转化成CHW型, 数字代表索引将要变化成为的值 
img_chw = img_chw[np.newaxis] #给CHW型图片加一个维度, 使之符合池化格式要求
input = torch.from_numpy(img_chw).float()
writer = SummaryWriter('logs')
output = ann(input)
writer.add_image('input',img_chw[0])
writer.add_image('output',output[0])
writer.close()


## 非线性激活

使用不同的函数对数据进行非线性变化, 产生一组新的张量, 可以提高神经网络的泛化性能

常用非线性激活函数:

ReLU

Sigmod等

*非线性激活实例*

In [ ]:
import torch.nn as nn
import cv2
import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter
#for example:
# nn.ReLU(inplace=False)
'''
    Args:
        Input: (*), where * means any number of dimensions. #输入可以为任意
        Output: (*), same shape as the input. #输出也为任意
        inplace = can optionally do the operation in-place. Default: False #是否将处理后的值覆盖原值, 默认不覆盖
'''

class Model(nn.Module):
    
    def __init__(self):
        super(Model,self).__init__()
        self.sigmoid = nn.Sigmoid() #使用Sigmoid激活函数

    def forward(self,input):
        output = self.sigmoid(input)
        return output

ann = Model()
img_path = 'bue_78.png'
img = cv2.imread(img_path)
img_chw = np.transpose(img,(2,0,1)) #将HWC格式的ndarrary图片,转化成CHW型, 数字代表索引将要变化成为的值 
img_chw = img_chw[np.newaxis] #给CHW型图片加一个维度, 使之符合格式要求
input = torch.from_numpy(img_chw)
output = ann(input)
writer = SummaryWriter('logs')
writer.add_image('input',img_chw[0])
writer.add_image('output',output[0])
writer.close()

## 正则化

In [ ]:
# torch.nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True, device=None, dtype=None)
'''
    Args:
        num_features = C from an expected input of size (N, C, H, W) #在输入四个size中的channel数目
        eps = a value added to the denominator for numerical stability. Default: 1e-5
        momentum = the value used for the running_mean and running_var computation. 
                Can be set to None for cumulative moving average (i.e. simple average). Default: 0.1
        affine = a boolean value that when set to True, this module has learnable affine parameters. Default: True
        track_running_stats = a boolean value that when set to True, this module tracks the running mean and variance, 
                        and when set to False, this module does not track such statistics, 
                        and initializes statistics buffers running_mean and running_var as None. 
                        When these buffers are None, this module always uses batch statistics. in both training and eval modes. 
                        Default: True
'''

## 线性层

In [ ]:
import numpy as np
import cv2
import torch.nn as nn
import torch
# torch.nn.Linear(in_features, out_features, bias=True, device=None, dtype=None)
'''
    Args:
        in_features = size of each input sample  #输入神经元个数
        out_features = size of each output sample #输出神经元个数
        bias = If set to False, the layer will not learn an additive bias. Default: True #是否具有偏置
'''

class Model(nn.Module):

    def __init__(self):
        super(Model,self).__init__()
        self.linear = nn.Linear(2021040,20)
    
    def forward(self,input):
        output = self.linear(input)
        return output
ann = Model()
img_path = 'bue_78.png'
img = cv2.imread(img_path)
img_chw = np.transpose(img,(2,0,1)) #将HWC格式的ndarrary图片,转化成CHW型, 数字代表索引将要变化成为的值 
img_chw = img_chw[np.newaxis] #给CHW型图片加一个维度, 使之符合格式要求
input = torch.from_numpy(img_chw).float()
input = torch.flatten(input) #使用展平函数将input展为一维数组
print(input.shape)
output = ann(input)
print(output.shape)

## nn.Sequential

用于在初始化中初始化一系列模型, 在后续前向算法调用时可以直接调用Sequential的实例而不需一个一个调用模型

In [5]:
import torch.nn as nn

class Model(nn.Module):
    
    def __init__(self):
        super(Model,self).__init__()
        self.model = nn.Sequential( #定义一个线性模型队列
            nn.Conv2d(3,32,5),
            nn.MaxPool2d(2),
            nn.Flatten(), #用于将数据展开为一维数据方便线性层处理
            nn.Linear(1024,32),
            nn.Linear(32,10)
        )
    
    def forward(self,input):
        output = self.model(input)
        return output
    
ann = Model()
print(ann)

Model(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Flatten(start_dim=1, end_dim=-1)
    (3): Linear(in_features=1024, out_features=32, bias=True)
    (4): Linear(in_features=32, out_features=10, bias=True)
  )
)


## Loss Function

有各种各样的损失函数, 具体使用查用PyTorch官网描述


使用时, 先创建类的实例, 再通过类的实例去传入output和label


In [ ]:
Loss = nn.MSEloss()

result = Loss(output,label)

## BackWard

对损失函数使用.backward()求梯度. 便于以后优化

In [ ]:
Loss = nn.MSEloss()

result = Loss(output,label)

result.backward()

## Optimizer 优化器

用于对模型中的参数进行调整

In [ ]:
from torch.utils.data import DataLoader
import torch.nn as nn
from torchvision import datasets,transforms
from torch import optim
from torch.utils.tensorboard import SummaryWriter

dataset = datasets.CIFAR10('dataset',train=False,transform=transforms.ToTensor(),download=True)
dataloader = DataLoader(dataset,batch_size=64)

class Model(nn.Module):

    def __init__(self):
        super(Model,self).__init__()
        self.NetualNet = nn.Sequential(
            nn.Conv2d(3,32,5,padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,32,5,padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64,5,padding=2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(1024,64),
            nn.Linear(64,10)
        )


    def forward(self,input):
        output = self.NetualNet(input)
        return output

ANN = Model()
writer = SummaryWriter('logs')
loss = nn.CrossEntropyLoss()
optim = optim.SGD(ANN.parameters(),lr=0.01) #创建一个优化器, 使用SGD优化(随机梯度下降)
for epoch in range(50):                    #一共训练50轮
    epoch_loss = 0.0                        #每轮损失函数值
    print('Now we are in epoch NO.{}'.format(epoch))
    i = 0
    for data in dataloader:                #数据集中图片分别加载
        imgs,labels = data
        output = ANN(imgs)
        result_loss = loss(output,labels)
        optim.zero_grad()                   #重要!!!!!!在进行优化前先将优化器梯度归零
        result_loss.backward()              #求梯度
        optim.step()                        #反向传播优化
        epoch_loss = epoch_loss + result_loss #每个图像的损失叠加
        print('Image number is {}'.format(i))
        print(f"This image's loss is {result_loss}")
        i = i + 1 
    writer.add_scalar('Epoch loss',epoch_loss,epoch)
writer.close()

## 现有模型修改

In [ ]:
import torch.nn as nn
import torchvision

vgg_add = torchvision.models.vgg16(weights=None) #加载已有模型
print(vgg_add) #打印已有模型结构

vgg_add.add_module('add',nn.Linear(1000,10)) #加入一个线性层在整个分类最外面
vgg_add.classifier.add_module('addTOclas',nn.Linear(1000,10)) #加入到classifier序列层中
print(vgg_add)

vgg_change = torchvision.models.vgg16(weights=None)

vgg_change.classifier[6] = nn.Linear(4096,10) #将vgg的classifier层中第[6]位层更改
print(vgg_change)

### 保存模型

In [ ]:
from turtle import forward
import torch.nn as nn
import torchvision
import torch

vgg16 = torchvision.models.vgg16(weights=None) #加载已有模型
  
torch.save(vgg16,'vgg16_mothed1.pt') #保存vgg16所有, 包括参数以及网络结构

torch.save(vgg16.state_dict(),'vgg16_mothed2.pt') #只保存vgg16参数为有序字典形式

class Model(nn.Module): #自定义神经网络

    def __init__(self) :
        super().__init__()
        self.conv = nn.Conv2d(3,64,3)

    def forward(self,input):
        output = self.conv(input)
        return output
ann = Model()
torch.save(ann,'mymodel.pt') #保存全模型

### 加载模型

In [ ]:
import torch 
import torchvision
from save_model import Model  #将自己的模型引入, 以免加载模型时出错


vgg = torch.load('vgg16_mothed1.pt') #加载第一种全方式保存的模型
print(vgg)

#第二种方式加载时, 一定要先创建模型, 然后再加载数值
new_vgg = torchvision.models.vgg16(weights=None)  #加载模型
new_vgg.load_state_dict(torch.load('vgg16_mothed2.pt')) # 加载参数
print(new_vgg)

#加载自己的模型时, 一定要先引入定义时的神经网络类, 
#随后才能正确加载, 否则将会报错
mymodel = torch.load('mymodel.pt')
print(mymodel)

## 利用GPU训练与训练完整套路


In [ ]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
#判断环境是否用GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#加载数据集
test_data = torchvision.datasets.CIFAR10('test_dataset',train=False,transform=torchvision.transforms.ToTensor(),
                                            download=True)
train_data = torchvision.datasets.CIFAR10('train_dataset',train=True,transform=torchvision.transforms.ToTensor(),
                                            download=True)
tran_loader = DataLoader(train_data,batch_size=64,shuffle=True)
test_loader = DataLoader(test_data,batch_size=64)

#得到并打印数据集大小
train_data_size = len(train_data)
test_data_size = len(test_data)
print(f'训练数据集大小为: {train_data_size}')
print(f'测试数据集大小为: {test_data_size}')

#引入TensorBoard记录数据
writer = SummaryWriter('logs')

#创建模型
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.mymodel = nn.Sequential(
            nn.Conv2d(3,32,5,padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,32,5,padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64,5,padding=2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(1024,64),
            nn.Linear(64,10)
        )
    
    def forward(self,input): 
        output = self.mymodel(input)
        return output 

#创建网络实例
ann = Model()
ann.to(device=device) #将神经网络移动到GPU上, 此步不需要重新赋值

#创建LossFunction(交叉熵)
loss_fn = nn.CrossEntropyLoss()
loss_fn.to(device) #将损失函数移动到GPU上, 此步不需要重新赋值

#创建优化器(随机梯度下降优化器)
learning_rate = 1e-2
optim = torch.optim.SGD(ann.parameters(),lr=learning_rate)

#训练过程
ann.train() #神经网络进入训练模式, 一些特殊的层需要先进入训练模式
epoch = 10
all_roll = 0
for i in range(epoch):

    all_loss = 0
    print(f'******现在在第 {i} 轮训练******')
    for data in tran_loader:
        imgs, labels = data
        #将训练数据放到GPU上,此处一定要重新赋值
        imgs = imgs.to(device)
        labels = torch.tensor(labels)
        labels = labels.to(device)
        #end
        output = ann(imgs)
        loss = loss_fn(output,labels)
        optim.zero_grad()
        loss.backward()
        optim.step()
        all_roll = all_roll + 1
        if all_roll % 100 == 0:
            print(f'训练轮次: {all_roll} ,损失: {loss}')
        all_loss = all_loss + loss
    print(f'本轮总训练损失：{all_loss.item()}')#去除Tensor数据类型, 直接打印数值
    writer.add_scalar('train_loss',all_loss,i) 
    #训练验证
    ann.eval() #神经网络进入验证模式
    with torch.no_grad(): #去除自动微分, 防止验证时干扰反向传播
        all_test_loss = 0
        all_accurate = 0
        for data in test_loader:
            imgs, labels = data 
            #迁移到GPU
            imgs = imgs.to(device)
            labels = torch.tensor(labels)
            labels = labels.to(device)

            out_test = ann(imgs)
            loss_test = loss_fn(out_test, labels)
            all_test_loss = all_test_loss + loss_test

            '''
                求准确率时用argmax()函数, 返回一个数组中最大的值的下标
                torch.argmax(input, dim, keepdim=False) → LongTensor
                Args:
                    input (Tensor) = the input tensor. #输入Tensor张量
                    dim (int) = the dimension to reduce. If None, the argmax of the flattened input is returned.
                                                        #求最大标的维数, 0是高, 1是宽
                    keepdim (bool) = whether the output tensor has dim retained or not. Ignored if dim=None.                                   
            '''
            accurate = torch.argmax(out_test,1)
            all_accurate =all_accurate + (accurate == labels).sum() #bollean表达式返回一个True(1)和False(0)列表, .sum()求和

        accurate_rate = all_accurate/test_data_size
        print(f'验证损失: {all_test_loss}')
        print(f'验证准确率为: {accurate_rate*100:.2f}%')
        writer.add_scalar('test_loss',all_test_loss,i)
        writer.add_scalar('accurate_rate',accurate_rate,i)
    #模型保存
    torch.save(ann.state_dict(),'LAST.pt') #最后一轮训练的参数模型
        #将用来保存损失最小的模型
    if i == 0:
        torch.save(ann.state_dict(),'BEST.pt')
        judge_loss = all_test_loss
    if all_test_loss < judge_loss:
        judge_loss = all_test_loss
        torch.save(ann.state_dict(),'BEST.pt')
        print(f'现在训练最好的轮数是: {i}')
writer.close()

## 神经网络模型的使用

In [ ]:
import torch.nn as nn
import cv2 
import numpy as np
import torch
from train import Model  #引入自己的模型

img = cv2.imread('horse1.jpg')
img = cv2.resize(img,(32,32))  #重置图片大小, 使之符合进模型条件格式
while 1:
    cv2.imshow('resize',img)
    c = cv2.waitKey() & 0xff
    if c == 27:
        break
cv2.destroyAllWindows()
img = np.transpose(img,(2,0,1))
img = img[np.newaxis,...] #重设图片维度, 使之符合格式
img = torch.from_numpy(img).float()
label_dict = {'airplane':0,'automobile':1,'bird':2,'cat':3,'deer':4,'dog':5,'frog':6,
                'horse':7,'ship':8,'truck':9}

weight = 'from_colab/BEST.pt'
model = Model()
model.load_state_dict(torch.load(weight,map_location='cpu')) #加载模型, map_location: 将模型从CUDA映射到CPU上

model.eval() # 设置模型测试模式
with torch.no_grad():
    output = model(img)
print(output)
for k,v in label_dict.items():
    if v == torch.argmax(output).item():
        print(k)
        break